# Analyse des Tendances Climatiques en France : Évolution des Variables Météorologiques de 1950 à 2023

L'analyse présentée dans ce notebook se concentre sur l'exploration des données météorologiques quotidiennes collectées par Météo France entre 1950 et 2023. Les données proviennent du fichier `Q_64_previous-1950-2023_RR-T-Vent.csv.gz`, disponible à l'adresse https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations détaillées sur les conditions météorologiques quotidiennes enregistrées dans divers postes météorologiques en France, notamment les précipitations, les températures minimales et maximales, ainsi que d'autres variables.

L'objectif principal de cette analyse est de calculer la somme cumulée des moyennes journalières de certaines variables météorologiques clés (précipitations, températures moyennes, maximales et minimales) pour chaque année, en se concentrant sur la valeur de fin d'année pour chaque variable. Cela permettra d'étudier l'évolution de ces variables sur le temps et d'identifier d'éventuelles tendances climatiques.


## Méthodologie

La méthodologie employée dans cette analyse repose sur l'utilisation de DuckDB pour traiter et analyser les données météorologiques. La première étape consiste à charger les données et à calculer les moyennes journalières pour les variables d'intérêt. Ensuite, une somme cumulée de ces moyennes est calculée pour chaque année. Les résultats sont finalement filtrés pour ne conserver que la valeur de fin d'année pour chaque variable, permettant ainsi d'étudier leur évolution annuelle. Les données résultantes sont ensuite visualisées à l'aide de Matplotlib et Seaborn pour mettre en évidence les tendances observées.


 
Les détails de la requête SQL utilisée pour l'analyse sont cruciaux pour comprendre les étapes de traitement. La requête commence par calculer les moyennes journalières pour les variables météorologiques, puis procède au calcul de la somme cumulée de ces moyennes pour chaque année. Finalement, seules les valeurs de fin d'année sont conservées pour chaque variable.


Une visualisation soignée est proposée pour illustrer les résultats de l'analyse. Les courbes de tendance pour les cumuls annuels de précipitations, températures moyennes, maximales et minimales sont tracées, avec un lissage sur 10 ans pour mettre en évidence les tendances à long terme. Les visualisations sont réalisées avec soin pour offrir une représentation claire et informative des données.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_64_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH daily_avg AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS annee,
    CAST(SUBSTRING("AAAAMMJJ", 5, 2) AS INTEGER) AS mois,
    CAST(SUBSTRING("AAAAMMJJ", 7, 2) AS INTEGER) AS jour,
    AVG(CAST("RR" AS DOUBLE)) AS avg_rr,
    AVG(CAST("TM" AS DOUBLE)) AS avg_tm,
    AVG(CAST("TX" AS DOUBLE)) AS avg_tx,
    AVG(CAST("TN" AS DOUBLE)) AS avg_tn
  FROM 
    loaded_dataset
  GROUP BY 
    annee, mois, jour
),
cum_sum AS (
  SELECT 
    annee,
    jour_de_annee,
    SUM(avg_rr) OVER (PARTITION BY annee ORDER BY jour_de_annee) AS cum_rr,
    SUM(avg_tm) OVER (PARTITION BY annee ORDER BY jour_de_annee) AS cum_tm,
    SUM(avg_tx) OVER (PARTITION BY annee ORDER BY jour_de_annee) AS cum_tx,
    SUM(avg_tn) OVER (PARTITION BY annee ORDER BY jour_de_annee) AS cum_tn
  FROM (
    SELECT 
      annee,
      (mois - 1) * 30 + jour AS jour_de_annee,
      avg_rr,
      avg_tm,
      avg_tx,
      avg_tn
    FROM 
      daily_avg
  ) AS subquery
)
SELECT 
  annee,
  cum_rr AS cum_rr_end_year,
  cum_tm AS cum_tm_end_year,
  cum_tx AS cum_tx_end_year,
  cum_tn AS cum_tn_end_year
FROM 
  cum_sum
WHERE 
  jour_de_annee = (SELECT MAX(jour_de_annee) FROM cum_sum AS cs WHERE cs.annee = cum_sum.annee)
ORDER BY 
  annee """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 74 lignes


,annee,cum_rr_end_year,cum_tm_end_year,cum_tx_end_year,cum_tn_end_year
0,1950,1354.693791,4973.35,6319.083333,2722.300000
1,1951,1537.841796,4766.50,5870.350000,2546.475000
2,1952,1583.263803,4852.20,6059.233333,2542.516667
3,1953,915.175000,4638.45,6016.910000,2443.906667
4,1954,1498.891898,4490.85,5693.094048,2441.085119


## 📈 Visualisation

La bibliothèque principale utilisée pour cette visualisation est Seaborn, une extension de Matplotlib, qui permet de créer des graphiques statistiques attractifs et informatifs. Ce choix est adapté car Seaborn offre une grande flexibilité pour personnaliser les visualisations et Matplotlib fournit une base solide pour les représentations graphiques. Cela permet une représentation claire et concise de l'évolution des composantes météorologiques cumulées par année.

In [4]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

palette = {
    "cum_rr_end_year": "#add8e6",   # light blue : pluie
    "cum_tm_end_year": "#ffd700",   # gold       : température moyenne
    "cum_tx_end_year": "#ff4500",   # orangered  : température max
    "cum_tn_end_year": "#4682b4"    # steelblue  : température min
}

# Lissage sur 10 ans (fenêtre glissante centrée, dropna pour nettoyer)
window = 10
df_plot = df.copy()
for col in ["cum_rr_end_year", "cum_tm_end_year", "cum_tx_end_year", "cum_tn_end_year"]:
    df_plot[f"{col}_smooth"] = df_plot[col].rolling(window, center=True).mean()

df_plot = df_plot.dropna(subset=[f"{col}_smooth" for col in palette.keys()])

sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "font.family": "DejaVu Sans",
    "font.size": 11,
})
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

labels = {
    "cum_rr_end_year": "Cumul précipitations (mm)",
    "cum_tm_end_year": "Cumul T moyenne (°C)",
    "cum_tx_end_year": "Cumul T max (°C)",
    "cum_tn_end_year": "Cumul T min (°C)",
}

for idx, col in enumerate(palette.keys()):
    ax = axes[idx]
    ax.plot(
        df_plot["annee"],
        df_plot[col],
        color=palette[col],
        linewidth=1.5,
        alpha=0.7,
        label="Données brutes (rouge clair)",
    )
    ax.plot(
        df_plot["annee"],
        df_plot[f"{col}_smooth"],
        color="blue",
        linewidth=2.5,
        label="Courbe lissée (10 ans)",
    )
    ax.set_title(labels[col], fontsize=14, weight="bold")
    ax.set_xlabel("Année")
    ax.set_ylabel("Valeur cumulée")
    ax.legend(frameon=False)
    ax.yaxis.set_major_formatter(StrMethodFormatter("{x:,.0f}"))
    sns.despine(ax=ax)

plt.suptitle("Évolution des composantes météorologiques cumulées par année", fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.97])
dataviz = fig
dataviz

ModuleNotFoundError: No module named 'seaborn'

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250820_223026.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250820_223026.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


AVERTISSEMENT: Aucune variable 'dataviz' trouvée.
